In [5]:
import os
import sys 
import json
import glob
import random
import collections
import time
import re
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

from torch import nn
from torch.utils import data as torch_data

from sklearn import model_selection as sk_model_selection
from torch.nn import functional as torch_functional
import torch.nn.functional as F

from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder


from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,roc_curve,mean_absolute_error,auc,confusion_matrix

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import SpectralClustering
import networkx as nx

import torch
from sklearn.model_selection import train_test_split



from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import pandas as pd

print('ok')


ok


In [3]:
def set_seed(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False  

set_seed(42)

In [4]:
df_1 = pd.read_csv('input-2/csv/labels_2classes.csv')
df_clinical_encoded = pd.read_excel('df_clinical_encoded.xlsx')
df_filtered = df_1[~df_1['patient_id'].isin(['p0297', 'p0298', 'p0299', 'p0303', 'p0307','p0305','p0301'])]          ##### No clinical data for these patients
df_filtered = df_filtered.reset_index(drop=True)
print(df_filtered)
patient_ids = df_filtered['patient_id'].values  # get patient_id 

# put patient_id  to the first column of  df_clinical_encoded 
df_clinical_encoded.insert(0, 'patient_id', patient_ids)


#### Create 10 fold list for all patients
num_folds = 10
patient_labels = df_filtered.groupby('patient_id')['label'].first().reset_index()
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)


patient_ids = patient_labels['patient_id'].values
patient_y = patient_labels['label'].values

#split patient ID
fold_indices = list(skf.split(patient_ids, patient_y))


patient_fold_mapping = {}
for fold, (train_idx, test_idx) in enumerate(fold_indices):
    test_patient_ids = patient_ids[test_idx]
    for pid in test_patient_ids:
        patient_fold_mapping[pid] = fold


patient_fold_mapping



    patient_id  label
0        p0001      1
1        p0006      0
2        p0008      0
3        p0013      0
4        p0014      0
..         ...    ...
114      p0291      0
115      p0294      1
116      p0302      0
117      p0306      0
118      p0308      0

[119 rows x 2 columns]


{'p0036': 0,
 'p0058': 0,
 'p0081': 0,
 'p0149': 0,
 'p0162': 0,
 'p0164': 0,
 'p0185': 0,
 'p0188': 0,
 'p0198': 0,
 'p0250': 0,
 'p0283': 0,
 'p0291': 0,
 'p0027': 1,
 'p0065': 1,
 'p0101': 1,
 'p0114': 1,
 'p0144': 1,
 'p0159': 1,
 'p0165': 1,
 'p0219': 1,
 'p0262': 1,
 'p0274': 1,
 'p0286': 1,
 'p0287': 1,
 'p0013': 2,
 'p0021': 2,
 'p0062': 2,
 'p0071': 2,
 'p0113': 2,
 'p0160': 2,
 'p0193': 2,
 'p0197': 2,
 'p0208': 2,
 'p0224': 2,
 'p0266': 2,
 'p0302': 2,
 'p0033': 3,
 'p0043': 3,
 'p0092': 3,
 'p0104': 3,
 'p0119': 3,
 'p0145': 3,
 'p0180': 3,
 'p0183': 3,
 'p0200': 3,
 'p0228': 3,
 'p0236': 3,
 'p0285': 3,
 'p0014': 4,
 'p0025': 4,
 'p0046': 4,
 'p0063': 4,
 'p0077': 4,
 'p0187': 4,
 'p0212': 4,
 'p0223': 4,
 'p0235': 4,
 'p0239': 4,
 'p0263': 4,
 'p0269': 4,
 'p0001': 5,
 'p0008': 5,
 'p0150': 5,
 'p0157': 5,
 'p0167': 5,
 'p0202': 5,
 'p0213': 5,
 'p0231': 5,
 'p0241': 5,
 'p0254': 5,
 'p0276': 5,
 'p0306': 5,
 'p0031': 6,
 'p0038': 6,
 'p0048': 6,
 'p0068': 6,
 'p0163': 6,

In [6]:

exclude_ids = ['p0297', 'p0298', 'p0299', 'p0303', 'p0307', 'p0305', 'p0301']

# Define a function to read, filter, and rename columns  , Patient_ID  //// patient_id 
def load_and_process(file_path):
    df = pd.read_csv(file_path)
    
    if 'Patient_ID' in df.columns:
        df = df.rename(columns={'Patient_ID': 'patient_id'})
    
    elif 'patient_id' not in df.columns:
        raise ValueError(f"{file_path} NO 'Patient_ID'  or  'patient_id' ")
    

    df_filtered = df[~df['patient_id'].isin(exclude_ids)]
    df_filtered = df_filtered.reset_index(drop=True)
    
    return df_filtered

# process the 3 image features
df_red = load_and_process('features_red.csv')
df_green = load_and_process('features_green.csv')
df_yellow = load_and_process('features_yellow.csv')



# delete 'red_features'、'green_features'  'yellow_features' （if already exist）
if 'red_features' in df_red.columns:
    df_red = df_red.drop(['red_features'], axis=1)
if 'green_features' in df_green.columns:
    df_green = df_green.drop(['green_features'], axis=1)
if 'yellow_features' in df_yellow.columns:
    df_yellow = df_yellow.drop(['yellow_features'], axis=1)

# create feature column
df_red['red_features'] = df_red.iloc[:, 1:].values.tolist()
df_green['green_features'] = df_green.iloc[:, 1:].values.tolist()
df_yellow['yellow_features'] = df_yellow.iloc[:, 1:].values.tolist()

# merge image features in one dataframe
df_images = pd.merge(df_red[['patient_id', 'red_features']], 
                     df_green[['patient_id', 'green_features']], 
                     on='patient_id', how='inner')
df_images = pd.merge(df_images, df_yellow[['patient_id', 'yellow_features']], 
                     on='patient_id', how='inner')

# merge all
df = pd.merge(df_clinical_encoded, df_images, on='patient_id', how='inner')
df = pd.merge(df, df_filtered[['patient_id', 'label']], on='patient_id', how='inner')

# Determine feature columns and label columns, and exclude unnecessary columns
clinical_feature_cols = [col for col in df.columns if col not in ['patient_id', 'label', 'red_features', 'green_features', 'yellow_features']]
label_col = 'label'


In [7]:
class ClinicalImageDataset(Dataset):
    def __init__(self, df, clinical_feature_cols, label_col):
        """
        Args:
            df (pd.DataFrame): 
            clinical_clinical_feature_cols (list): 
            label_col (str): 
        """
        # clinical feature
        self.X_clinical = torch.tensor(df[clinical_feature_cols].values, dtype=torch.float32)
        
        # image features
        self.X_image_red = torch.from_numpy(np.stack(df['red_features'].values)).float()  # (B, 512)
        self.X_image_green = torch.from_numpy(np.stack(df['green_features'].values)).float()  # (B, 512)
        self.X_image_yellow = torch.from_numpy(np.stack(df['yellow_features'].values)).float()  # (B, 512)
        
        # label
        self.y = torch.tensor(df[label_col].values, dtype=torch.float32)
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return (self.X_clinical[idx],
                self.X_image_red[idx],
                self.X_image_green[idx],
                self.X_image_yellow[idx],
                self.y[idx])

# # 3. define  simple cross attention
# class CrossAttentionModule(nn.Module):
#     def __init__(self, embed_dim=512, num_heads=8, dropout=0.1):
#         super(CrossAttentionModule, self).__init__()
#         self.cross_attn = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, dropout=dropout, batch_first=True)
#         self.layer_norm1 = nn.LayerNorm(embed_dim)
#         self.feed_forward = nn.Sequential(
#             nn.Linear(embed_dim, embed_dim * 2),
#             nn.ReLU(),
#             nn.Linear(embed_dim * 2, embed_dim)
#         )
#         self.layer_norm2 = nn.LayerNorm(embed_dim)
#         self.dropout = nn.Dropout(dropout)
    
#     def forward(self, queries, keys, values):
#         """
#         Args:
#             queries: (B, Q_len, E) - 
#             keys: (B, K_len, E) - 
#             values: (B, K_len, E) - 
#         Returns:
#             out2: (B, Q_len, E) - 
#             attn_weights: (B, Q_len, K_len) 
#         """
#         attn_output, attn_weights = self.cross_attn(queries, keys, values)  # (B, Q_len, E), (B, Q_len, K_len)
#         attn_output = self.dropout(attn_output)
#         out1 = self.layer_norm1(queries + attn_output) 
        
#         ff_output = self.feed_forward(out1)  # (B, Q_len, E)
#         ff_output = self.dropout(ff_output)
#         out2 = self.layer_norm2(out1 + ff_output)  
        
#         return out2, attn_weights

### mutual cross attention
class MutualCrossAttentionModule(nn.Module):
    def __init__(self, embed_dim=512, num_heads=8, dropout=0.1):
        super(MutualCrossAttentionModule, self).__init__()
        self.mha = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, dropout=dropout, batch_first=True)
        self.layer_norm1 = nn.LayerNorm(embed_dim)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * 2),
            nn.ReLU(),
            nn.Linear(embed_dim * 2, embed_dim)
        )
        self.layer_norm2 = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x1, x2):
        """
        x1: (B, len1, E)
        x2: (B, len2, E)
        
        Mutual:
        1. x1=Q, x2 as K,V, get output_A
        2. x2=Q, x1 as K,V,g et output_B
        output = output_A + output_B
        
        
        """
        # x1->x2
        output_A, attn_weights_A = self.mha(x1, x2, x2)  # (B, len1, E), (B, len1, len2)
        # x2->x1
        output_B, attn_weights_B = self.mha(x2, x1, x1)  # (B, len2, E), (B, len2, len1)

        
        output = output_A + output_B  # (B, len1, E)

      
        out1 = self.layer_norm1(x1 + output)
        ff_output = self.feed_forward(out1)
        ff_output = self.dropout(ff_output)
        out2 = self.layer_norm2(out1 + ff_output)

       
        return out2, attn_weights_A


# 4. define fusion model
class ClinicalImageFusionModel(nn.Module):
    def __init__(self, clinical_dim=85, embed_dim=512, num_heads=4, dropout=0.1):
        super(ClinicalImageFusionModel, self).__init__()
        # clinical MLP: 85 -> 512
        self.clinical_mlp = nn.Sequential(
            nn.Linear(clinical_dim, 256),
            nn.LeakyReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, embed_dim)
        )
        
        # cross attention
        self.cross_attn = MutualCrossAttentionModule(embed_dim=embed_dim, num_heads=num_heads, dropout=dropout)
        
        # classifier
        self.classifier = nn.Sequential(
            nn.Linear(embed_dim, 256),
            nn.LeakyReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, 1)  # logit output
        )
    
    def forward(self, clinical_features, image_red, image_green, image_yellow):
        """
        Args:
            clinical_features: (B, 85)
            image_red: (B, 512)
            image_green: (B, 512)
            image_yellow: (B, 512)
        Returns:
            logits: (B, 1)
            attn_weights: (B, 3, 3)
        """
        # encode clinical
        clinical_embed = self.clinical_mlp(clinical_features)  # (B, 512)
        
        # copy 3  clinical times
        clinical_embed = clinical_embed.unsqueeze(1).repeat(1, 3, 1)  # (B, 3, 512)
        
        # merge image features
        image_features = torch.stack([image_red, image_green, image_yellow], dim=1)  # (B, 3, 512)
        
        # cross attetion 
        fused_features, attn_weights = self.cross_attn(clinical_embed, image_features)  # (B, 3, 512), (B, 3, 3)
        
        # merge fusion features
        fused_features = fused_features.mean(dim=1)  # (B, 512)
        
        # classifier 
        logits = self.classifier(fused_features)  # (B, 1)
        
        return logits, attn_weights


In [8]:
def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for X_clinical, X_red, X_green, X_yellow, y_batch in dataloader:
        # to GPU
        X_clinical = X_clinical.to(device)
        X_red = X_red.to(device)
        X_green = X_green.to(device)
        X_yellow = X_yellow.to(device)
        y_batch = y_batch.to(device)
        y_batch = y_batch.float()

        optimizer.zero_grad()
        outputs, _ = model(X_clinical, X_red, X_green, X_yellow)  # [B, 1]
        outputs = outputs.squeeze(1)  # [B]
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * X_clinical.size(0)
        
        #  train ACC
        predicted = (torch.sigmoid(outputs) > 0.36).long()
        correct += (predicted == y_batch.long()).sum().item()
        total += y_batch.size(0)
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

def validate_auc(model, dataloader, device):
    model.eval()
    all_targets = []
    all_outputs = []
    with torch.no_grad():
        for X_clinical, X_red, X_green, X_yellow, y_batch in dataloader:
            #  to GPU
            X_clinical = X_clinical.to(device)
            X_red = X_red.to(device)
            X_green = X_green.to(device)
            X_yellow = X_yellow.to(device)
            y_batch = y_batch.to(device)

            outputs, _ = model(X_clinical, X_red, X_green, X_yellow)  # [B, 1]
            outputs = outputs.squeeze(1)
            probas = torch.sigmoid(outputs).cpu().numpy()  
            all_outputs.append(probas)
            all_targets.append(y_batch.cpu().numpy())

    all_outputs = np.concatenate(all_outputs)
    all_targets = np.concatenate(all_targets)
    auc = roc_auc_score(all_targets, all_outputs)
    return auc

def test_inference(model, dataloader, device):
    model.eval()
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for X_clinical, X_red, X_green, X_yellow, y_batch in dataloader:
          
            X_clinical = X_clinical.to(device)
            X_red = X_red.to(device)
            X_green = X_green.to(device)
            X_yellow = X_yellow.to(device)

            outputs, _ = model(X_clinical, X_red, X_green, X_yellow)  # [B, 1]
            outputs = outputs.squeeze(1)  # [B]
            probas = torch.sigmoid(outputs).cpu().numpy()  
            all_preds.append(probas)
            all_targets.append(y_batch.numpy())
    all_preds = np.concatenate(all_preds)
    all_targets = np.concatenate(all_targets)
    return all_targets, all_preds


In [9]:

# hyper parameters 
num_folds = 10
batch_size = 16
learning_rate = 1e-4
num_epochs = 100
patience = 10#  early stopping

# train  (10-fold)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

all_y_test = []
all_y_pred_proba = []

for fold in range(num_folds):
    print(f"Processing fold {fold + 1}/{num_folds}...")
    
    test_fold = fold
    val_fold = (fold - 1) % num_folds 
    train_folds = [i for i in range(num_folds) if i != test_fold and i != val_fold]
    
    train_patient_ids = [pid for pid, f in patient_fold_mapping.items() if f in train_folds]
    val_patient_ids = [pid for pid, f in patient_fold_mapping.items() if f == val_fold]
    test_patient_ids = [pid for pid, f in patient_fold_mapping.items() if f == test_fold]
    
    fold_ids = {
        "train_ids": train_patient_ids,
        "val_ids": val_patient_ids,
        "test_ids": test_patient_ids
    }
    print(f"Fold {fold + 1} IDs: {fold_ids}")
    
    # split data
    df_train = df[df['patient_id'].isin(train_patient_ids)].copy()
    df_valid = df[df['patient_id'].isin(val_patient_ids)].copy()
    df_test = df[df['patient_id'].isin(test_patient_ids)].copy()
    
    # norm only for clinical data
    scaler = StandardScaler()
    scaler.fit(df_train[clinical_feature_cols])   # fit on train dataset 
    
    df_train[clinical_feature_cols] = scaler.transform(df_train[clinical_feature_cols])
    df_valid[clinical_feature_cols] = scaler.transform(df_valid[clinical_feature_cols])
    df_test[clinical_feature_cols] = scaler.transform(df_test[clinical_feature_cols])
    
    # create dataset
    train_dataset = ClinicalImageDataset(df_train, clinical_feature_cols, label_col)
    val_dataset = ClinicalImageDataset(df_valid, clinical_feature_cols, label_col)
    test_dataset = ClinicalImageDataset(df_test, clinical_feature_cols, label_col)
    
    # dataloader
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    # model
    clinical_dim = len(clinical_feature_cols)  # 85
    embed_dim = 512     # embedding dim
    num_heads = 4
    dropout = 0.2
    
    model = ClinicalImageFusionModel(clinical_dim=clinical_dim,
                                     embed_dim=embed_dim,
                                     num_heads=num_heads,
                                     dropout=dropout).to(device)
    
    # loss and optimizer
    criterion = nn.BCEWithLogitsLoss()
    optimizer = Adam(model.parameters(), lr=learning_rate)
    
    # train
    no_improve_count = 0
    best_val_auc = 0.0
    best_model_weights = None
    
    for epoch in range(num_epochs):
        train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
        val_auc = validate_auc(model, val_loader, device)
        print(f"Fold[{fold+1}] Epoch [{epoch+1}/{num_epochs}] Train Loss: {train_loss:.4f} Train Acc: {train_acc:.4f}, Val AUC: {val_auc:.4f}")
    
        if val_auc > best_val_auc:
            best_val_auc = val_auc
            best_model_weights = model.state_dict()
            no_improve_count = 0
        else:
            no_improve_count += 1
    
        if no_improve_count >= patience:
            print("Validation AUC hasn't improved for 5 epochs. Early stopping...")
            break
    
    # load best model
    if best_model_weights is not None:
        model.load_state_dict(best_model_weights)
    
    # inference in test data
    y_test_fold, y_pred_proba_fold = test_inference(model, test_loader, device)
    all_y_test.append(y_test_fold)
    all_y_pred_proba.append(y_pred_proba_fold)
    
    print(f"Fold[{fold+1}] completed.\n")

# collect all test 
all_y_test = np.concatenate(all_y_test)
all_y_pred_proba = np.concatenate(all_y_pred_proba)


best_threshold = 0.36
all_y_pred = np.where(all_y_pred_proba > best_threshold, 1, 0)

 ###confusion matrix
tn, fp, fn, tp = confusion_matrix(all_y_test, all_y_pred).ravel()
acc = accuracy_score(all_y_test, all_y_pred)
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0 
f1 = f1_score(all_y_test, all_y_pred) 
mae = mean_absolute_error(all_y_test, all_y_pred)
auc = roc_auc_score(all_y_test, all_y_pred_proba)


print("all test label:", all_y_test)
print("all test probability:", all_y_pred_proba)
print(f"\noverall AUC: {auc:.4f}")
print(f"overall Sensitivity (Recall): {sensitivity:.4f}")
print(f"overall Specificity: {specificity:.4f}")
print(f"overall F1 Score: {f1:.4f}")
print(f"overall Accuracy: {acc:.4f}")
print(f"overall MAE: {mae:.4f}")


Processing fold 1/10...
Fold 1 IDs: {'train_ids': ['p0027', 'p0065', 'p0101', 'p0114', 'p0144', 'p0159', 'p0165', 'p0219', 'p0262', 'p0274', 'p0286', 'p0287', 'p0013', 'p0021', 'p0062', 'p0071', 'p0113', 'p0160', 'p0193', 'p0197', 'p0208', 'p0224', 'p0266', 'p0302', 'p0033', 'p0043', 'p0092', 'p0104', 'p0119', 'p0145', 'p0180', 'p0183', 'p0200', 'p0228', 'p0236', 'p0285', 'p0014', 'p0025', 'p0046', 'p0063', 'p0077', 'p0187', 'p0212', 'p0223', 'p0235', 'p0239', 'p0263', 'p0269', 'p0001', 'p0008', 'p0150', 'p0157', 'p0167', 'p0202', 'p0213', 'p0231', 'p0241', 'p0254', 'p0276', 'p0306', 'p0031', 'p0038', 'p0048', 'p0068', 'p0163', 'p0175', 'p0192', 'p0242', 'p0257', 'p0272', 'p0284', 'p0288', 'p0006', 'p0023', 'p0051', 'p0099', 'p0106', 'p0118', 'p0125', 'p0186', 'p0189', 'p0217', 'p0238', 'p0290', 'p0024', 'p0026', 'p0056', 'p0132', 'p0139', 'p0182', 'p0203', 'p0207', 'p0216', 'p0221', 'p0226', 'p0251'], 'val_ids': ['p0017', 'p0040', 'p0153', 'p0166', 'p0195', 'p0232', 'p0253', 'p0268', 